In [ ]:
#!pip install konlpy
#!pip install kss --use-feature=fast-deps


In [3]:
import pandas as pd
# Konlpy 라이브러리 설치

# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Konlpy 라이브러리 불러오기
from konlpy.tag import Okt
okt = Okt()

# 필요한 라이브러리 불러오기
import re
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import kss
from konlpy.tag import Okt


'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [4]:
import glob
import os

### csv파일 데이터프레임으로 읽기

crawling_naverblog/blog_crawled.csv 파일을 
preprocessing_naverblog/temp.py를 사용하여 맞춤법 검사 및 클렌징을 한 후
preprocessing_naverblog/blog_clean_data.csv를 만들어서 사용함.
colab환경에서 hanspell library가 돌아가지 않아 그렇게 진행함.

In [7]:
# 파일 위치
file = 'blog_clean_data.csv'
df = pd.read_csv(file)

In [8]:

#잘 생성됐는지 확인
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     4000 non-null   object
 1   link      4000 non-null   object
 2   contents  4000 non-null   object
dtypes: object(3)
memory usage: 93.9+ KB


,title,link,contents
0,[헝가리 교환학생] D+117 우울은 갑자기 찾아와,https://blog.naver.com/sypark0330/222733061710,무슨 큰일이 있었던 건 아니지만 아닌가 큰일인가 하여튼 급속도로 우울해져버렸다 오...
1,[노르웨이 교환학생] Day+11: 뻐스투어의 즐거움 뭉크의 우울함과 첫 한식의 반가움,https://blog.naver.com/cindy7760/223319977078,오늘은 버스투어 사실 간 장소들은 잘 기억 안 난다 오슬로에서 제일 큰 공원 스키...
2,캐나다 교환학생 | 9월 3주차 : 우울과 행복 그 사이 어디쯤,https://blog.naver.com/hyevely2000/222899258235,캐나다 교환학생 월 주차 에브리데이 일상 오히려 좋아에서 결제했다...
3,"[헝가리 교환학생] 우울과 행복사이, 피크닉 4월 일상 그리고 헝가리 미용실",https://blog.naver.com/jennadiary/223443936131,지금도 런던 스탠스테드 공항에서 시간 기다리며 블로그를 살짝 건드려 보아오 런...
4,"일본 교환학생 | 한국 돌아가고 싶어요, 독감 걸리면 우울해지나요, 가을 타는지...",https://blog.naver.com/slsxpseh_/223326407375,月日 친구가 일하는 스타방 방문 유학생 체험학습 파티 때 도우미로 왔던 일본인 친...


In [26]:
# 결측치 확인
is_null = (df['contents'] == '').sum()
is_null = is_null + (df['contents'] == ' ').sum()
is_null = is_null + (df['contents'] == '  ').sum()
is_null = is_null + (df['contents'] == '   ').sum()
is_null = is_null + (df['contents'] == '    ').sum()
is_null = is_null + (df['contents'] == '     ').sum()

is_null

939

In [27]:
# 결측값 채우기: 맞춤법 검사 안 된 걸로

df_temp = pd.read_csv('../crawling_naverblog/blog_crawled.csv')

df['contents'] = df['contents'].replace('', pd.NA)
df['contents'] = df['contents'].replace(' ', pd.NA)
df['contents'] = df['contents'].replace('  ', pd.NA)
df['contents'] = df['contents'].replace('   ', pd.NA)
df['contents'] = df['contents'].replace('    ', pd.NA)
df['contents'] = df['contents'].replace('     ', pd.NA)

df['contents'].fillna(df_temp['contents'], inplace=True)

# 전처리
#### cleaned_content 구하기 (문장 불용어 제거, 전처리, 명사 남기기)
#### filtering (우울, 불안, 스트레스 없는 문장 제거)

In [31]:
from konlpy.tag import Okt
okt = Okt()

In [32]:
from tqdm import tqdm

In [33]:

file_path = 'preprocessing_blog.csv'

# 필요 함수 정의
def filter_sentences(text):
    keywords = ['우울', '불안', '스트레스']
    sentences = text.split('.') 
    filtered_sentences = [sentence for sentence in sentences if any(keyword in sentence for keyword in keywords)]
    return ' '.join(filtered_sentences)

def pos_filtering(text):
    pos_list = ['Noun']
    words = okt.pos(text, stem=True)
    filtered_words = [word[0] for word in words if word[1] in pos_list]
    return filtered_words

def remove_stopwords(words, stopwords):
    return [word for word in words if word not in stopwords]

def preprocess_text(text, stopwords):
    sentences = kss.split_sentences(text)
    processed_sentences = []
    for sentence in sentences:
        if any(keyword in sentence for keyword in ['우울', '불안', '스트레스']):
            filtered_words = pos_filtering(sentence)
            cleaned_sentence = remove_stopwords(filtered_words, stopwords)
            processed_sentences.append(cleaned_sentence)
    return processed_sentences

# 불용어 목록 로드
with open('../stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

In [35]:
# 다음 1000개의 데이터를 처리하고 싶은 경우, range의 시작점을 변경
start_index = 0  # 이전에 처리한 마지막 인덱스 + 1
batch_size = 4000  # 한 번에 처리할 데이터의 개수

file_path = 'blog_preprocessing.csv'

# 데이터프레임을 지정된 범위에서 분할하여 처리
results = []
for start in tqdm(range(start_index, start_index + batch_size, batch_size), desc="Processing in batches"):
    end = start + batch_size
    subset = df.iloc[start:end]  # batch_size만큼의 행을 선택
    subset['contents'] = subset['contents'].apply(filter_sentences)
    subset['preprocessed'] = subset['cleaned_content'].apply(lambda text: preprocess_text(text, stopwords))
    results.append(subset)

# 결과를 하나의 데이터프레임으로 합치기
final_df = pd.concat(results)

# 파일 존재 여부 확인 후 적절한 헤더 설정 및 저장
if os.path.exists(file_path):
    header_exists = True
else:
    header_exists = False

final_df.to_csv(file_path, mode='a', index=False, header=not header_exists)

Processing in batches:   0%|          | 0/1 [05:10<?, ?it/s]


KeyboardInterrupt: 